# Pomer dát 8:2

Súbor: train_8_test_2.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 
from sklearn.model_selection import train_test_split

Načítanie dát

In [3]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

In [7]:
X = np.concatenate([x_train,x_test])

In [8]:
Y = np.concatenate([y_train,y_test])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [10]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(37993, 128, 128, 1)
(9499, 128, 128, 1)
(37993, 2)
(9499, 2)


CNN

In [11]:
model = Sequential()

In [12]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [13]:
model.add(Conv2D(64, 3, strides=1, activation='relu')) 

In [14]:
model.add(MaxPooling2D(3, 2))

In [15]:
model.add(Conv2D(64, 5, strides=1, padding='same', activation='relu'))

In [16]:
model.add(MaxPooling2D(3, 2))

In [17]:
model.add(Conv2D(64, 3, strides=1, padding='same', activation='relu'))

In [18]:
model.add(Conv2D(64, 3, strides=1, padding='same', activation='relu'))

In [19]:
model.add(MaxPooling2D(3, 2))

In [20]:
model.add(Flatten())

In [21]:
model.add(Dense(96, activation='relu'))

In [22]:
model.add(Dropout(0.25)) 

In [23]:
model.add(Dense(96, activation='relu'))

In [24]:
model.add(Dropout(0.25)) 

In [25]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [26]:
adam = Adam()

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [28]:
saved_model = "model_8_2.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

test loss je chyba klasifikácie (FN + FP) / súčet všetkých

test accuracy je úspešnosť klasifikácie (TP + TN) / súčet všetkých

In [29]:
f = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 37993 samples, validate on 9499 samples
Epoch 1/20
37993/37993 [==============================] - 1281s 34ms/step - loss: 0.3658 - accuracy: 0.8322 - val_loss: 0.2356 - val_accuracy: 0.9075

Epoch 00001: val_accuracy improved from -inf to 0.90746, saving model to model_8_2.hdf5
Epoch 2/20
37993/37993 [==============================] - 1341s 35ms/step - loss: 0.2091 - accuracy: 0.9207 - val_loss: 0.1637 - val_accuracy: 0.9488

Epoch 00002: val_accuracy improved from 0.90746 to 0.94884, saving model to model_8_2.hdf5
Epoch 3/20
37993/37993 [==============================] - 1259s 33ms/step - loss: 0.1385 - accuracy: 0.9541 - val_loss: 0.1130 - val_accuracy: 0.9641

Epoch 00003: val_accuracy improved from 0.94884 to 0.96410, saving model to model_8_2.hdf5
Epoch 4/20
37993/37993 [==============================] - 1264s 33ms/step - loss: 0.1113 - accuracy: 0.9635 - val_loss: 0.0985 - val_accuracy: 0.9700

Epoch 00004: val_accuracy improved from 0.96410 to 0.97000, saving model to 

Načítanie modelu

In [30]:
print("Loading model....")
model = load_model('model_8_2.hdf5')

Loading model....


Vyhodnotenie test loss a accuracy

In [31]:
#na trénovacej množine 
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04176461377963053
Test accuracy: 0.9869712591171265


In [32]:
#na testovacej množine
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 0.09395373942561276
Test accuracy: 0.9802084565162659


Predikcia

In [34]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

Confusion Matrix

In [35]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [36]:
# TN / FN
# FP / TP
print(matrix)

[[4728   24]
 [ 164 4583]]


Classification Report

In [37]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      4752
           1       0.99      0.97      0.98      4747

   micro avg       0.98      0.98      0.98      9499
   macro avg       0.98      0.98      0.98      9499
weighted avg       0.98      0.98      0.98      9499
 samples avg       0.98      0.98      0.98      9499



In [33]:
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 31, 64)       